In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder

# --- 1. 加载和基础清理 ---
columns = [
    'age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 
    'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'num'
]
df = pd.read_csv('processed.cleveland.data', header=None, names=columns, na_values='?')

# 用众数填充'ca'和'thal'的缺失值
for col in ['ca','thal']:
    mode_val = df[col].mode()[0]
    df[col] = df[col].fillna(mode_val)

# --- 2. 统一的、正确的预处理流程 ---

# 复制一份原始数据框用于处理，避免修改原始df
df_processed = df.copy()

# a) 对连续值进行分箱
continuous_cols = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']
for col in continuous_cols:
    df_processed[col] = pd.cut(df_processed[col], bins=4, labels=False, include_lowest=True)

# b) 对其他已经是分类的列进行统一编码
# 这些列不是从0开始，或者值不连续，需要LabelEncoder
categorical_cols = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal']
for col in categorical_cols:
    le = LabelEncoder()
    # 使用完整的、未划分的数据进行学习和转换
    df_processed[col] = le.fit_transform(df_processed[col])

# --- 3. 准备最终的 X 和 y ---
X = df_processed.drop('num', axis=1)
y = df_processed['num']

# 关键：确保所有数据都是整数类型，以防万一
X = X.astype(int)
y = y.astype(int) # 目标变量也最好是整数

print("--- 预处理完成后的数据预览 ---")
print(X.head())
print("--- 各列的数据类型和唯一值数量 ---")
for col in X.columns:
    print(f"列 '{col}': Dtype={X[col].dtype}, 唯一值={X[col].unique()}")


# --- 4. 划分数据集 (使用分层抽样) ---
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y  # 强烈建议使用分层抽样
)

# --- 5. 训练和预测 ---
model = CategoricalNB()
model.fit(X_train, y_train)

# 现在预测一定能成功
y_pred = model.predict(X_test)

print("🎉🎉🎉 预测成功！🎉🎉🎉")

# --- 6. 评估模型 ---
accuracy = accuracy_score(y_test, y_pred)
print(f"模型准确率: {accuracy:.4f}")
print("分类报告:")
print(classification_report(y_test, y_pred))
print("混淆矩阵:")
print(confusion_matrix(y_test, y_pred))



--- 预处理完成后的数据预览 ---
   age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  slope  \
0    2    1   0         1     0    1        2        2      0        1      2   
1    3    1   3         2     1    0        2        1      1        0      1   
2    3    1   3         0     0    0        2        1      1        1      1   
3    0    1   2         1     1    0        0        3      0        2      2   
4    0    0   1         1     0    0        2        3      0        0      0   

   ca  thal  
0   0     1  
1   3     0  
2   2     2  
3   0     0  
4   0     0  
--- 各列的数据类型和唯一值数量 ---
列 'age': Dtype=int64, 唯一值=[2 3 0 1]
列 'sex': Dtype=int64, 唯一值=[1 0]
列 'cp': Dtype=int64, 唯一值=[0 3 2 1]
列 'trestbps': Dtype=int64, 唯一值=[1 2 0 3]
列 'chol': Dtype=int64, 唯一值=[0 1 2 3]
列 'fbs': Dtype=int64, 唯一值=[1 0]
列 'restecg': Dtype=int64, 唯一值=[2 0 1]
列 'thalach': Dtype=int64, 唯一值=[2 1 3 0]
列 'exang': Dtype=int64, 唯一值=[0 1]
列 'oldpeak': Dtype=int64, 唯一值=[1 0 2 3]
列 'slope': Dtype=int64

D:\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
D:\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
D:\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
